In [ ]:
pip install langchain langgraph openai pymupdf

In [ ]:
pip install langchain-community

In [85]:
import fitz
import os
import openai
from openai import OpenAI
from dotenv import load_dotenv
import langgraph
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage
from langgraph.graph import StateGraph

In [87]:
load_dotenv('/content/config.env')

openai_api_key = os.getenv("OPENAI_API_KEY")

os.environ["OPENAI_API_KEY"] = openai_api_key

client = OpenAI(api_key=openai_api_key )

In [38]:
from google.colab import files
uploaded = files.upload()

Saving Lakshani_Galwatta_CV.pdf to Lakshani_Galwatta_CV.pdf


In [88]:
# Define Job Requirement
JOB_REQUIREMENTS = """
We are looking for a Software Engineer with:
- Strong experience in Python and JavaScript.
- Familiarity with Machine Learning and Data Science.
- Knowledge of cloud services (AWS, Azure, or GCP).
- At least 3 years of experience in software development.
"""

In [89]:
# Function to extract text from a PDF
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

In [99]:
#Function to analyze resume's content and compares it against job requirements
def analyze_resume_with_openai(resume_text, job_requirements):
    messages = [
        {"role": "system", "content": "You are an AI assistant trained to evaluate resumes for job suitability."},
        {"role": "system", "content": f"Job Requirements: {job_requirements}"},
        {"role": "user", "content": f"Resume Content: {resume_text}"},
        {"role": "system", "content": "Please evaluate how well the resume matches the job requirements."}
    ]

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        temperature=1,
        messages=messages
    )

    return response.choices[0].message.content


In [100]:
# Function to classify resume based on analysis
def classify_resume_with_openai(analysis):
    messages = [
        {"role": "system", "content": "You are an AI assistant trained to classify resumes based on suitability for a job."},
        {"role": "system", "content": f"Analysis of Resume: {analysis}"},
        {"role": "system", "content": "Classify the candidate into one of the following categories: STRONG MATCH: If they meet all key requirements., NEEDS IMPROVEMENT: If they partially meet the requirements., or NOT SUITABLE: If they lack most skills."},
        {"role": "system", "content": " Do not include any explanations or additional details; respond with only one of these categories."}
    ]

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        temperature=1,
        messages=messages
    )

    return response.choices[0].message.content


In [104]:
from typing import Dict, TypedDict, Optional
from langgraph.graph import StateGraph
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage

# Define the state structure using TypedDict
class ResumeState(TypedDict):
    resume_text: Optional[str] = None
    analysis: Optional[str] = None
    classification: Optional[str] = None
    job_requirements: Optional[str] = None


# Define the Resume Screening Graph
graph = StateGraph(ResumeState)

# Step 1: Analyze Resume
def analyze_resume(state: ResumeState):
    print("🔹 Analyzing Resume...")
    analysis = analyze_resume_with_openai(state["resume_text"], state["job_requirements"])
    return {"analysis": analysis}

# Step 2: Classify Candidate
def classify_resume(state: ResumeState):
    print("🔹 Classifying Candidate...")
    classification = classify_resume_with_openai(state["analysis"])
    return {"classification": classification}

# Add nodes to the graph
graph.add_node("analyze_resume", analyze_resume)
graph.add_node("classify_resume", classify_resume)

# Define workflow connections (set sequence of operations)
graph.add_edge("analyze_resume", "classify_resume")

# Set the entry point (starting node)
graph.set_entry_point("analyze_resume")

# Compile the graph to create the application
app = graph.compile()


In [ ]:
# Function to run the resume screening process
def screen_resume(pdf_path: str, job_requirements: str)  -> Dict[str, str]:
    print("🔹 Extracting text from resume PDF...")
    resume_text = extract_text_from_pdf(pdf_path)

    initial_state = ResumeState(resume_text=resume_text, job_requirements=job_requirements)
    print(f"🔹 Initial State Created:{initial_state}")

    # Run the graph (invoke the workflow)
    final_state = app.invoke(initial_state)

    return {
        "Analysis": final_state["analysis"],
        "Classification": final_state["classification"],
    }

In [106]:
# Example usage: Run the resume screening process
if __name__ == "__main__":
    resume_file = "/content/Lakshani_Galwatta_CV.pdf"
    print("🔹 Running Resume Screening on:")
    result = screen_resume(resume_file, JOB_REQUIREMENTS)
    print("🔹 Resume Analysis:\n", result["Analysis"])
    print("\n🔹 Candidate Classification:", result["Classification"])

🔹 Running Resume Screening on:
🔹 Extracting text from resume PDF...
🔹 Initial State Created:{'resume_text': 'Lakshani Galwatta\nSoftware Engineer\nI am an enthusiastic, self motivated and hard working individual and I am keen to work for your reputable organization where\nI can demonstrate my abilities while enhancing my skills, to achieve the company\'s goals on time and with excellence.\n+94 77 574 07 57\nlakshani.galwatta@gmail.com\nlakshani-galwatta\nPython Engineer at Cove Island Industries Pvt Ltd\nContributed to the UrsaLeo cloud base digital twin solution system.\nConducted POC to integrate Things Board for real-time sensor data visualization.\nImproved the RAG chatbot backend to answer user queries enhancing response relevance and optimizing response\ntime.\nConducted R&D to implement a RAG chatbot using ColPali and  GPT-4o models.\nConducted R&D to identify optimal in-house multimodal model for chatbot development.\nDesigned and implemented web based user interface for chatbo